Imports

In [ ]:
print('imports')

Task 1: Generate puzzles

1.1 define puzzle representations

using a 2D array , matrix grid where it is NxN grid size. 

(A-Z) is for the region labels
( * ) is for when a star is present in the grid


In [ ]:
#example with 5x5 grid 1 star per row/column/region
puzzle = [
 ['A', 'B', 'C', 'C', 'C'],
 ['A', 'B', 'C', 'C', 'D'],
 ['A', 'A', 'C', 'D', 'D'],
 ['E', 'C', 'C', 'D', 'D'],
 ['E', 'E', 'E', 'D', 'D']
 ]

solution = [
 ['0', '1', '0', '0', '0'],
 ['0', '0', '0', '1', '0'],
 ['1', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '1'],
 ['0', '0', '1', '0', '0']
]

Similar for 10x10 grid with 2 stars per row/column/region

example with labels

In [ ]:
print("example with labels")

1.2 Constraint representations

each row contains only k stars
each column contains only k stars
each region contains only k stars
no two stars can be adjacent

pre compute constraint maps

In [ ]:
print("constraint mapping examples")

1.3 Data structure choices

In [ ]:
print("examples class")

1.4 Storing and compiling puzzles

1.4 SAT solve to validate uniqueness of puzzles

1.6 Generate initial test set of puzzles

In [ ]:
print("test set of puzzles")